In [ ]:
# On peut générer une page web qui ne contient que l'affichage avec voila, qui est installé
# dans l'environnement : 
# voila notebook.ipynb

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider, Layout, HBox, VBox, Checkbox, Select
from IPython.display import display
from scipy.interpolate import interp1d, PchipInterpolator 

# --- Configuration Matplotlib ---
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Segoe UI Emoji', 'Apple Color Emoji', 'Segoe UI Symbol', 'Twemoji', 'Noto Color Emoji']
plt.rcParams['mathtext.fontset'] = 'custom' 
# ------------------------------

# --- 1. Définition des fonctions de base ---

def signal_original(t, f1, f2):
    """Signal analogique original (somme de deux sinusoïdes) avec fréquences f1 et f2."""
    return np.sin(2 * np.pi * f1 * t) + 0.5 * np.sin(2 * np.pi * f2 * t)

# --- 2. Fonctions d'Interpolation (AVEC LIMITATION N_USED) ---

def limit_samples(t_samples, x_samples, N_used):
    """Limite les échantillons utilisés aux N_used premiers."""
    N_to_use = min(N_used, len(x_samples))
    t_limited = t_samples[:N_to_use]
    x_limited = x_samples[:N_to_use]
    return t_limited, x_limited

def interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used):
    """Interpolateur d'Ordre Zéro (ZOH) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) == 0:
        return np.zeros_like(t_analog)
        
    f_interp = interp1d(t_limited, x_limited, kind='previous', bounds_error=False, fill_value=(x_limited[0], x_limited[-1]))
    x_reconstruit = f_interp(t_analog)
    
    # Mise à zéro après le dernier échantillon utilisé
    if len(t_limited) > 0:
        x_reconstruit[t_analog > t_limited[-1]] = 0.0
        
    return x_reconstruit

def interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used):
    """Interpolateur d'Ordre Un (FOH / Linéaire) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) < 2:
        # Fallback au ZOH si FOH n'est pas possible
        return interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used)
        
    f_interp = interp1d(t_limited, x_limited, kind='linear', bounds_error=False, fill_value=(x_limited[0], x_limited[-1]))
    x_reconstruit = f_interp(t_analog)
    
    # Mise à zéro après le dernier échantillon utilisé
    if len(t_limited) > 0:
        x_reconstruit[t_analog > t_limited[-1]] = 0.0
        
    return x_reconstruit

def interpolate_pchip(t_analog, t_samples, x_samples, order, N_used, Fs):
    """Interpolateur Lisse (PCHIP - Ordre 3) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) < 4:
        # Fallback au FOH si PCHIP n'est pas possible (4 points requis)
        return interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used)
        
    f_interp = PchipInterpolator(t_limited, x_limited, extrapolate=True)
    x_reconstruit = f_interp(t_analog)
    
    last_sample_time = t_limited[-1]
    
    # Mise à zéro après le dernier échantillon utilisé
    x_reconstruit[t_analog > last_sample_time] = 0.0
    
    # ZOH avant le premier échantillon (pour la causalité)
    x_reconstruit[t_analog < t_limited[0]] = x_limited[0]
    
    return x_reconstruit

def reconstruction_sinc_full(t_analog, t_samples, x_samples, Fs, N_used):
    """Reconstruction Sinc (non-causal, limitée par N_used)."""
    x_reconstruit = np.zeros_like(t_analog)
    Ts = 1 / Fs
    
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    for n_index in range(len(t_limited)):
        sinc_contribution = x_limited[n_index] * np.sinc((t_analog - t_limited[n_index]) / Ts)
        x_reconstruit += sinc_contribution
    
    return x_reconstruit

# --- 3. Fonction interactive de traçage ---

def plot_reconstruction_universal(f1, f2, Fs, T_max, interpolator_type, N_used, show_sincs):
    
    # 3.1 Préparation des données
    t_analog = np.linspace(0, T_max, 1000) 
    Ts = 1 / Fs        
    n_samples_max = int(T_max * Fs)
    n = np.arange(n_samples_max) 
    
    if n_samples_max < 1:
        print("⚠️ Avertissement: Fs ou T_max trop faibles.")
        return

    t_samples = n * Ts      
    x_samples = signal_original(t_samples, f1, f2) 

    x_reconstruit = np.zeros_like(t_analog)
    sinc_contributions = []

    # 3.2 Interpolation selon le type choisi
    if N_used == 0:
        x_reconstruit = np.zeros_like(t_analog)

    elif interpolator_type == 'Sinc (Idéal)':
        x_reconstruit = reconstruction_sinc_full(t_analog, t_samples, x_samples, Fs, N_used)
        
        if show_sincs:
             t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
             for n_index in range(len(t_limited)):
                sinc_contribution = x_limited[n_index] * np.sinc((t_analog - t_limited[n_index]) / Ts)
                sinc_contributions.append(sinc_contribution)
    
    elif interpolator_type == 'ZOH (Ordre 0)':
        x_reconstruit = interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used)
        
    elif interpolator_type == 'FOH (Ordre 1)':
        x_reconstruit = interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used)

    elif interpolator_type == 'Ordre 3 (Lisse)':
        x_reconstruit = interpolate_pchip(t_analog, t_samples, x_samples, order=3, N_used=N_used, Fs=Fs)
        
    
    # 3.3 Calculs et Plotting
    x_analog_original = signal_original(t_analog, f1, f2)
    f_max = max(f1, f2)
    f_nyquist = 2 * f_max
    reconstruction_error = np.abs(x_analog_original - x_reconstruit)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # Tracé des éléments principaux
    if interpolator_type == 'Sinc (Idéal)' and show_sincs:
        for sc in sinc_contributions:
            ax1.plot(t_analog, sc, color='gray', linestyle='--', linewidth=1, alpha=0.3)
            
    # Limiter l'affichage des échantillons à N_used (pour le stem)
    t_plot_samples, x_plot_samples = limit_samples(t_samples, x_samples, N_used)
            
    ax1.plot(t_analog, x_analog_original, label='Signal Analogique Original', color='gray', linestyle='--')
    ax1.stem(t_plot_samples, x_plot_samples, linefmt='b-', markerfmt='bo', basefmt=" ", label=f'Échantillons numériques utilisés: {len(t_plot_samples)}')
    ax1.plot(t_analog, x_reconstruit, label=f'Signal Reconstruit ({interpolator_type})', color='r', linewidth=2)
    
    ax1.set_title(f"Interpolation ({interpolator_type}). Fs={Fs} Hz. Nyquist: {f_nyquist:.1f} Hz. Durée: {T_max:.1f} s", fontsize=14)
    ax1.set_ylabel("Amplitude")
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle=':', alpha=0.6)
    ax1.set_ylim(-2.2, 2.2)

    # Indicateur de statut (corrigé contre UnboundLocalError)
    text_status = f"Interpolateur: {interpolator_type}"
    text_color = 'black'

    if Fs < f_nyquist and interpolator_type == 'Sinc (Idéal)':
        text_status = f"⚠️ Aliasing : Fs < {f_nyquist:.1f} Hz"
        text_color = 'red'
    elif Fs >= f_nyquist and interpolator_type == 'Sinc (Idéal)':
        text_status = f"✅ Fs >= {f_nyquist:.1f} Hz"
        text_color = 'green'
    elif interpolator_type in ['ZOH (Ordre 0)', 'FOH (Ordre 1)', 'Ordre 3 (Lisse)']: 
        text_status = f"Causal ({interpolator_type.split('(')[0].strip()}) | Fs={Fs} Hz."
        text_color = 'blue'
    
    ax1.text(t_analog[-1]*0.05, 1.8, text_status, color=text_color, fontsize=12, bbox=dict(facecolor='white', alpha=0.7, edgecolor=text_color))
        
    ax2.plot(t_analog, reconstruction_error, color='purple', linewidth=1.5, label='Erreur Absolue |Original - Reconstruit|')
    ax2.set_xlabel("Temps (s)")
    ax2.set_ylabel("Erreur Absolue")
    ax2.grid(True, linestyle=':', alpha=0.6)
    ax2.set_ylim(bottom=0)
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# --- 4. Création des Widgets Interactifs ---

largeur_widget = '500px'
style = {'description_width': 'initial'}
layout_wide = Layout(width=largeur_widget)

f1_slider = FloatSlider(min=0.5, max=10.0, step=0.5, value=5.0, description='Fréquence f1 (Hz):', style=style, layout=layout_wide)
f2_slider = FloatSlider(min=0.5, max=5.0, step=0.5, value=1.0, description='Fréquence f2 (Hz):', style=style, layout=layout_wide)

Fs_slider = IntSlider(min=4, max=50, step=1, value=15, description='Fréquence Échant. Fs (Hz):', style=style, layout=layout_wide)

# MISE À JOUR DU LIBELLÉ
N_max_slider = IntSlider(min=1, max=60, step=1, value=10, description="Nbre d'Échantillons Utilisés:", style=style, layout=layout_wide)

T_max_slider = FloatSlider(min=1.0, max=8.0, step=0.5, value=4.0, description='Durée du Signal T_max (s):', style=style, layout=layout_wide)
sinc_visibility_checkbox = Checkbox(value=False, description='Afficher les sincs individuelles (Sinc Idéal seulement)', style=style, layout=Layout(width='auto'))

interpolator_select = Select(
    options=['Sinc (Idéal)', 'ZOH (Ordre 0)', 'FOH (Ordre 1)', 'Ordre 3 (Lisse)'],
    value='Sinc (Idéal)',
    description='Interpolateur:',
    style=style,
    layout=Layout(width='300px')
)


# --- 5. Fonction de mise à jour dynamique pour les limites du slider N_used ---
def update_n_max(change):
    current_T_max = T_max_slider.value
    current_Fs = Fs_slider.value
    new_max = int(current_T_max * current_Fs)
    
    if new_max < 1: new_max = 1
         
    N_max_slider.max = new_max
    
    if N_max_slider.value > new_max:
        N_max_slider.value = new_max
        
Fs_slider.observe(update_n_max, names='value') 
T_max_slider.observe(update_n_max, names='value')
update_n_max(None)


# --- 6. Affichage ---

interactive_plot = interactive(
    plot_reconstruction_universal, 
    f1=f1_slider, 
    f2=f2_slider, 
    Fs=Fs_slider, 
    T_max=T_max_slider,
    interpolator_type=interpolator_select, 
    N_used=N_max_slider,
    show_sincs=sinc_visibility_checkbox
)

widgets_box = VBox([
    HBox([f1_slider, f2_slider]),
    HBox([Fs_slider, N_max_slider]),
    HBox([T_max_slider, interpolator_select, sinc_visibility_checkbox])
])

display(widgets_box, interactive_plot.children[-1])

Output()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider, Layout, HBox, VBox, Checkbox, Select
from IPython.display import display
from scipy.interpolate import interp1d, PchipInterpolator 

# --- Configuration Matplotlib ---
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Segoe UI Emoji', 'Apple Color Emoji', 'Segoe UI Symbol', 'Twemoji', 'Noto Color Emoji']
plt.rcParams['mathtext.fontset'] = 'custom' 
# ------------------------------

# --- 1. Définition des fonctions de base ---

def signal_original(t, f1, f2):
    """Signal analogique original (somme de deux sinusoïdes) avec fréquences f1 et f2."""
    return np.sin(2 * np.pi * f1 * t) + 0.5 * np.sin(2 * np.pi * f2 * t)

# --- 2. Fonctions d'Interpolation (AVEC LIMITATION N_USED) ---

def limit_samples(t_samples, x_samples, N_used):
    """Limite les échantillons utilisés aux N_used premiers."""
    N_to_use = min(N_used, len(x_samples))
    t_limited = t_samples[:N_to_use]
    x_limited = x_samples[:N_to_use]
    return t_limited, x_limited

def interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used):
    """Interpolateur d'Ordre Zéro (ZOH) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) == 0:
        return np.zeros_like(t_analog)
        
    f_interp = interp1d(t_limited, x_limited, kind='previous', bounds_error=False, fill_value=(x_limited[0], x_limited[-1]))
    x_reconstruit = f_interp(t_analog)
    
    if len(t_limited) > 0:
        x_reconstruit[t_analog > t_limited[-1]] = 0.0
        
    return x_reconstruit

def interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used):
    """Interpolateur d'Ordre Un (FOH / Linéaire) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) < 2:
        return interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used)
        
    f_interp = interp1d(t_limited, x_limited, kind='linear', bounds_error=False, fill_value=(x_limited[0], x_limited[-1]))
    x_reconstruit = f_interp(t_analog)
    
    if len(t_limited) > 0:
        x_reconstruit[t_analog > t_limited[-1]] = 0.0
        
    return x_reconstruit

def interpolate_pchip(t_analog, t_samples, x_samples, order, N_used, Fs):
    """Interpolateur Lisse (PCHIP - Ordre 3) limité par N_used."""
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    if len(t_limited) < 4:
        return interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used)
        
    f_interp = PchipInterpolator(t_limited, x_limited, extrapolate=True)
    x_reconstruit = f_interp(t_analog)
    
    last_sample_time = t_limited[-1]
    
    x_reconstruit[t_analog > last_sample_time] = 0.0
    x_reconstruit[t_analog < t_limited[0]] = x_limited[0]
    
    return x_reconstruit

def reconstruction_sinc_full(t_analog, t_samples, x_samples, Fs, N_used):
    """Reconstruction Sinc (non-causal, limitée par N_used)."""
    x_reconstruit = np.zeros_like(t_analog)
    Ts = 1 / Fs
    
    t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
    
    for n_index in range(len(t_limited)):
        sinc_contribution = x_limited[n_index] * np.sinc((t_analog - t_limited[n_index]) / Ts)
        x_reconstruit += sinc_contribution
    
    return x_reconstruit

# --- 3. Fonction interactive de traçage ---

def plot_reconstruction_universal(f1, f2, Fs, T_max, interpolator_type, N_used, show_sincs):
    
    # 3.1 Préparation des données
    t_analog = np.linspace(0, T_max, 1000) 
    Ts = 1 / Fs        
    n_samples_max = int(T_max * Fs)
    n = np.arange(n_samples_max) 
    
    if n_samples_max < 1:
        print("⚠️ Avertissement: Fs ou T_max trop faibles.")
        return

    t_samples = n * Ts      
    x_samples = signal_original(t_samples, f1, f2) 

    x_reconstruit = np.zeros_like(t_analog)
    sinc_contributions = []

    # 3.2 Interpolation selon le type choisi
    if N_used == 0:
        x_reconstruit = np.zeros_like(t_analog)

    elif interpolator_type == 'Sinc (Idéal)':
        x_reconstruit = reconstruction_sinc_full(t_analog, t_samples, x_samples, Fs, N_used)
        
        if show_sincs:
             t_limited, x_limited = limit_samples(t_samples, x_samples, N_used)
             for n_index in range(len(t_limited)):
                sinc_contribution = x_limited[n_index] * np.sinc((t_analog - t_limited[n_index]) / Ts)
                sinc_contributions.append(sinc_contribution)
    
    elif interpolator_type == 'ZOH (Ordre 0)':
        x_reconstruit = interpolate_zoh(t_analog, t_samples, x_samples, Fs, N_used)
        
    elif interpolator_type == 'FOH (Ordre 1)':
        x_reconstruit = interpolate_foh(t_analog, t_samples, x_samples, Fs, N_used)

    elif interpolator_type == 'Ordre 3 (Lisse)':
        x_reconstruit = interpolate_pchip(t_analog, t_samples, x_samples, order=3, N_used=N_used, Fs=Fs)
        
    
    # 3.3 Calculs et Plotting
    x_analog_original = signal_original(t_analog, f1, f2)
    f_max = max(f1, f2)
    f_nyquist = 2 * f_max
    reconstruction_error = np.abs(x_analog_original - x_reconstruit)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # Tracé des éléments principaux
    if interpolator_type == 'Sinc (Idéal)' and show_sincs:
        for sc in sinc_contributions:
            ax1.plot(t_analog, sc, color='gray', linestyle='--', linewidth=1, alpha=0.3)
            
    # Limiter l'affichage des échantillons à N_used (pour le stem)
    t_plot_samples, x_plot_samples = limit_samples(t_samples, x_samples, N_used)
            
    ax1.plot(t_analog, x_analog_original, label='Signal Analogique Original', color='gray', linestyle='--')
    ax1.stem(t_plot_samples, x_plot_samples, linefmt='b-', markerfmt='bo', basefmt=" ", label=f'Échantillons numériques utilisés: {len(t_plot_samples)}')
    ax1.plot(t_analog, x_reconstruit, label=f'Signal Reconstruit ({interpolator_type})', color='r', linewidth=2)
    
    ax1.set_title(f"Interpolation ({interpolator_type}). Fs={Fs} Hz. Nyquist: {f_nyquist:.1f} Hz. Durée: {T_max:.1f} s", fontsize=14)
    ax1.set_ylabel("Amplitude")
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle=':', alpha=0.6)
    ax1.set_ylim(-2.2, 2.2)

    # Indicateur de statut
    text_status = f"Interpolateur: {interpolator_type}"
    text_color = 'black'

    if Fs < f_nyquist and interpolator_type == 'Sinc (Idéal)':
        text_status = f"⚠️ Aliasing : Fs < {f_nyquist:.1f} Hz"
        text_color = 'red'
    elif Fs >= f_nyquist and interpolator_type == 'Sinc (Idéal)':
        text_status = f"✅ Fs >= {f_nyquist:.1f} Hz"
        text_color = 'green'
    elif interpolator_type in ['ZOH (Ordre 0)', 'FOH (Ordre 1)', 'Ordre 3 (Lisse)']: 
        text_status = f"Causal ({interpolator_type.split('(')[0].strip()}) | Fs={Fs} Hz."
        text_color = 'blue'
    
    ax1.text(t_analog[-1]*0.05, 1.8, text_status, color=text_color, fontsize=12, bbox=dict(facecolor='white', alpha=0.7, edgecolor=text_color))
        
    ax2.plot(t_analog, reconstruction_error, color='purple', linewidth=1.5, label='Erreur Absolue |Original - Reconstruit|')
    ax2.set_xlabel("Temps (s)")
    ax2.set_ylabel("Erreur Absolue")
    ax2.grid(True, linestyle=':', alpha=0.6)
    ax2.set_ylim(bottom=0)
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# --- 4. Création des Widgets Interactifs ---

largeur_widget = '500px'
style = {'description_width': 'initial'}
layout_wide = Layout(width=largeur_widget)

f1_slider = FloatSlider(min=0.5, max=10.0, step=0.5, value=5.0, description='Fréquence f1 (Hz):', style=style, layout=layout_wide)
f2_slider = FloatSlider(min=0.5, max=5.0, step=0.5, value=1.0, description='Fréquence f2 (Hz):', style=style, layout=layout_wide)

Fs_slider = IntSlider(min=4, max=50, step=1, value=15, description='Fréquence Échant. Fs (Hz):', style=style, layout=layout_wide)

N_max_slider = IntSlider(min=1, max=60, step=1, value=10, description="Nbre d'Échantillons Utilisés:", style=style, layout=layout_wide)

T_max_slider = FloatSlider(min=1.0, max=8.0, step=0.5, value=4.0, description='Durée du Signal T_max (s):', style=style, layout=layout_wide)

sinc_visibility_checkbox = Checkbox(
    value=True, 
    description='Afficher les sincs individuelles', 
    style=style, 
    layout=Layout(width='auto')
)

interpolator_select = Select(
    options=['Sinc (Idéal)', 'ZOH (Ordre 0)', 'FOH (Ordre 1)', 'Ordre 3 (Lisse)'],
    value='Sinc (Idéal)',
    description='Interpolateur:',
    style=style,
    layout=Layout(width='300px')
)

# --- FONCTION D'OBSERVATION (avec correction pour l'initialisation) ---
def toggle_sinc_checkbox(change):
    """Affiche/Masque la case à cocher sinc selon l'interpolateur sélectionné."""
    # current_value est accessible via change.new si l'événement vient de l'observation.
    current_value = change.new
        
    if current_value == 'Sinc (Idéal)':
        sinc_visibility_checkbox.layout.visibility = 'visible'
        sinc_visibility_checkbox.value = True
    else:
        sinc_visibility_checkbox.layout.visibility = 'hidden'
        sinc_visibility_checkbox.value = False
        
interpolator_select.observe(toggle_sinc_checkbox, names='value')

# --- INITIALISATION MANUELLE CORRIGÉE ---
# Utilisation de la logique d'affichage sans l'objet "change"
if interpolator_select.value == 'Sinc (Idéal)':
    sinc_visibility_checkbox.layout.visibility = 'visible'
else:
    sinc_visibility_checkbox.layout.visibility = 'hidden'
# ----------------------------------------


# --- 5. Fonction de mise à jour dynamique pour les limites du slider N_used ---
def update_n_max(change):
    current_T_max = T_max_slider.value
    current_Fs = Fs_slider.value
    new_max = int(current_T_max * current_Fs)
    
    if new_max < 1: new_max = 1
         
    N_max_slider.max = new_max
    
    if N_max_slider.value > new_max:
        N_max_slider.value = new_max
        
Fs_slider.observe(update_n_max, names='value') 
T_max_slider.observe(update_n_max, names='value')
update_n_max(None)


# --- 6. Affichage ---

interactive_plot = interactive(
    plot_reconstruction_universal, 
    f1=f1_slider, 
    f2=f2_slider, 
    Fs=Fs_slider, 
    T_max=T_max_slider,
    interpolator_type=interpolator_select, 
    N_used=N_max_slider,
    show_sincs=sinc_visibility_checkbox
)

widgets_box = VBox([
    HBox([f1_slider, f2_slider]),
    HBox([Fs_slider, N_max_slider]),
    HBox([T_max_slider, interpolator_select, sinc_visibility_checkbox]) 
])

display(widgets_box, interactive_plot.children[-1])

Output()